In [1]:
import os
import numpy as np
import pandas as pd
from math import sqrt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from scipy.stats import uniform, randint
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, GridSearchCV
import xgboost as xgb
from xgboost import plot_tree
import matplotlib.pyplot as plt

from IPython.display import display, HTML
display(HTML("<style>.container { width:55% !important; }</style>"))

# Data

In [2]:
dataset_df = pd.read_json("./data/all_ads.json")
dataset_df = dataset_df.drop(["id", "category"], axis=1)
dataset_df.head(5)

,fuel_cb,gearbox_cb,locality,manufacturer_cb,model_cb,price,tachometer,vehicle_age
0,Nafta,Automatická,Středočeský kraj,BMW,Řada 3,1349000,26000,1
1,Nafta,Manuální,Hlavní město Praha,BMW,Řada 3,455000,193107,5
2,Nafta,None,Jihočeský kraj,BMW,Řada 3,1510000,12000,1
3,Nafta,Automatická,Královéhradecký kraj,BMW,Řada 3,1228149,25400,1
4,Hybridní,Automatická,Moravskoslezský kraj,BMW,Řada 3,1703000,24000,1


In [3]:
df_skoda = dataset_df[dataset_df.manufacturer_cb == "Škoda"]
df_skoda = df_skoda.drop("manufacturer_cb", axis=1)
df_skoda.head()

,fuel_cb,gearbox_cb,locality,model_cb,price,tachometer,vehicle_age
106,Benzín,Manuální,Jihočeský kraj,Fabia,440900,0,0
107,Benzín,Manuální,Jihočeský kraj,Fabia,440900,0,0
108,Benzín,Manuální,Jihočeský kraj,Fabia,440900,0,0
109,Benzín,Manuální,Jihočeský kraj,Fabia,505600,0,0
110,Benzín,Manuální,Jihočeský kraj,Fabia,404100,0,0


In [4]:
score_df = df_skoda.copy()
onehot_columns = ['fuel_cb', 'gearbox_cb', 'locality', 'model_cb']
onehot_df = score_df[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns=onehot_columns, prefix_sep='-')
score_onehot_drop = score_df.drop(onehot_columns, axis=1)
score_onehot = pd.concat([score_onehot_drop, onehot_df], axis=1)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(score_onehot.drop(['price'], axis=1), score_onehot.price, test_size=0.20, random_state=42)

# Model & Training

In [6]:
# params = {
#     'learning_rate': (0.1,),
#     'subsample'    : (0.2,),
#     'n_estimators' : (500,),
#     'max_depth'    : (28,),
# }

# xgb_model = GradientBoostingRegressor()
# xgb_search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, verbose=1, n_jobs=2, return_train_score=True)

params = {
    'max_depth': [8, 10],
    'n_estimators': [400, 500]
}
xgb_model = xgb.XGBRegressor()
xgb_search = GridSearchCV(xgb_model, params, verbose=1, n_jobs=2)

In [7]:
%%time
xgb_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
CPU times: user 25 s, sys: 4.2 s, total: 29.2 s
Wall time: 1min 10s


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             n_jobs=2,
             param_grid={'max_depth': [8, 10], 'n_estimators': [400, 500]},
             verbose=1)

In [8]:
y_pred = xgb_search.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

RMSE: 68495.85350728169


# Show me the results!

In [13]:
def undummify(df, prefix_sep="-"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [14]:
pd.options.display.float_format = '{:.0f}'.format

df_results = pd.DataFrame({"predicted": y_pred, "real": y_test})
df_results["diff"] = df_results.real - df_results.predicted
df_results["diff_abs"] = abs(df_results.real - df_results.predicted)
df_results["diff_abs_round"] = round(abs(df_results.real - df_results.predicted), -3)
df_results.drop(["diff", "diff_abs"], axis=1, inplace=True)
df_results = df_results.merge(undummify(X_test), left_index=True, right_index=True).head(2000)
df_results.head(20)

,predicted,real,diff_abs_round,tachometer,vehicle_age,fuel_cb,gearbox_cb,locality,model_cb
66943,584365,616000,32000,24715,2,Benzín,Automatická,Hlavní město Praha,Kamiq
65224,740914,679900,61000,23328,1,Benzín,Manuální,Středočeský kraj,Karoq
50177,453500,485000,31000,169558,6,Nafta,Automatická,Hlavní město Praha,Superb
18394,519250,438000,81000,44022,2,Benzín,Automatická,Hlavní město Praha,Octavia
24374,708092,749999,42000,129000,4,Nafta,Automatická,Karlovarský kraj,Kodiaq
36201,1072511,1199000,126000,11290,3,Nafta,Automatická,Středočeský kraj,Kodiaq
12967,254142,340000,86000,85545,6,Benzín,Manuální,Hlavní město Praha,Rapid
42333,647790,659000,11000,7174,0,Benzín,Manuální,Hlavní město Praha,Kamiq
71997,181613,176000,6000,184662,8,Nafta,Manuální,Hlavní město Praha,Rapid
7185,380382,379000,1000,93995,5,Nafta,Manuální,Středočeský kraj,Octavia


In [15]:
# worst
df_results.sort_values(by='diff_abs_round', ascending=False).head(20)

,predicted,real,diff_abs_round,tachometer,vehicle_age,fuel_cb,gearbox_cb,locality,model_cb
48180,870944,1389000,518000,42556,1,Nafta,Automatická,Jihomoravský kraj,Kodiaq
6461,961456,1455500,494000,16325,1,Nafta,Automatická,Plzeňský kraj,Superb
2151,710757,1159900,449000,11965,2,Nafta,Automatická,Plzeňský kraj,Karoq
18115,967857,1350000,382000,26361,2,Nafta,Automatická,Hlavní město Praha,Kodiaq
22718,1255123,1586000,331000,0,0,Benzín,Automatická,Hlavní město Praha,Superb
65396,655861,344800,311000,133428,0,Nafta,Manuální,Moravskoslezský kraj,Octavia
41542,589076,889000,300000,21696,2,Nafta,Automatická,Kraj Vysočina,Octavia
54479,892459,1189000,297000,6434,1,Nafta,Automatická,Pardubický kraj,Kodiaq
13739,846220,1133800,288000,12000,1,Benzín,Automatická,Hlavní město Praha,Octavia
793,967706,684000,284000,44961,2,Benzín,Automatická,Hlavní město Praha,Kodiaq


In [16]:
# best
df_results.sort_values(by='diff_abs_round', ascending=True).head(30)

,predicted,real,diff_abs_round,tachometer,vehicle_age,fuel_cb,gearbox_cb,locality,model_cb
16569,277903,278000,0,53621,3,Benzín,Manuální,Hlavní město Praha,Fabia
38799,219952,220000,0,113489,8,Benzín,Manuální,Hlavní město Praha,Rapid
46584,47393,47000,0,172700,18,Benzín,Manuální,Ústecký kraj,Fabia
29381,35049,35000,0,192000,21,Benzín,Manuální,Ústecký kraj,Fabia
65386,459366,459000,0,165000,3,Nafta,Automatická,Jihomoravský kraj,Octavia
53036,491723,491600,0,1,0,Benzín,Manuální,Plzeňský kraj,Scala
54685,299137,299000,0,188000,8,Nafta,Automatická,Středočeský kraj,Octavia
7186,379349,379000,0,127533,4,Nafta,Manuální,Středočeský kraj,Octavia
19413,498032,498000,0,28556,1,Benzín,Manuální,Moravskoslezský kraj,Scala
51384,113510,114000,0,144200,13,Benzín,Manuální,Moravskoslezský kraj,Fabia


In [17]:
df_results.real[df_results.diff_abs_round <= 20000].count()

716

In [18]:
df_results.real.count()

2000